# Baku or pasar

## Setup

In [93]:
# YOUR EDIT HERE #
# Remember, keep your auth token to yourself!
# If you want to store this notebook in github, remember to remove your token first!
auth_token = "maio_cf75a54b_d24c_4ca3_b9c2_4b90d9d61d6f"

In [94]:
import requests

url = "https://competitions.aiolympiad.my/api/maio_2025/maio_2025_baku_or_pasar"

def post_answer(data: dict):
    response = requests.post(url=url, json=data, headers={"X-API-Key": auth_token})
    if response.status_code == 200:
        return response.json()
    else:
        return f"Failed to submit, status code is {response.status_code}\n{response.text}"

## Introduction

You are given two files, `train.jsonl` and `test.jsonl`.

In [95]:
import json

In [96]:
import urllib.request

urllib.request.urlretrieve(
    "https://storage.googleapis.com/aiolympiadmy/baku-or-pasar/train.jsonl", 
    "train.jsonl"
)
urllib.request.urlretrieve(
    "https://storage.googleapis.com/aiolympiadmy/baku-or-pasar/test.jsonl", 
    "test.jsonl"
)

('test.jsonl', <http.client.HTTPMessage at 0x1b9d67c0080>)

In [97]:
with open("train.jsonl", "r") as f:
    train = [json.loads(i) for i in f]

In [98]:
train[:5]

[{'id': 0,
  'text': 'cerai balik cenggitu jaaa, kaknab akan ganti tmpat som, dan kaknab lah perangai yg paling tk semegah jht dari som 😏',
  'class': 3},
 {'id': 1,
  'text': 'Pasukan Polis Marin (PPM)\xa0Wilayah Satu Langkawi\xa0membongkar\xa0sindiket penyeludupan minyak diesel ke Thailand menggunakan bot barter trade di Jeti Perikanan Lembaga Kemajuan Ikan Malaysia (LKIM) Kuala Perlis dekat sini semalam.',
  'class': 0},
 {'id': 2,
  'text': 'Sebanyak enam buah khemah peniaga bazar Ramadan di Alor Pongsu, Bagan Serai rosak akibat dibadai angin ribut kira-kira pukul 2.30 petang tadi.',
  'class': 0},
 {'id': 3,
  'text': 'Mas, di tempat kami ada hamparan batu di aliran sungai, hamparan batu tersebut menyelimuti batang pohon yg sebagian telah mengkristal. Fenomena yg demikian kira2 prosesnya bgmana yah.',
  'class': 2},
 {'id': 4,
  'text': 'Gempa bumi kuat berukuran 7.6 pada skala Richter menggegarkan Papua New Guinea dan mengorbankan lima orang, manakala sebilangan yang lain cedera,

From inspecting the first five items, you see that you are provided with a list of texts with an associated class. Bahasa baku is tagged with class 0, and bahasa pasar is tagged with classes 1 to 5.

In [99]:
with open("test.jsonl", "r") as f:
    test = [json.loads(i) for i in f]

In [100]:
test[:5]

[{'id': 0,
  'text': 'SALAH SATU CHANNEL YOUTUBE BELAJAR TOEFL GRATIS TERBAEKKK‼️\n\nGUYS YANG MAU PERSIAPAN TES TOEFL, KE CHANNEL INI AJAA, BAGUSSS, LENGKAP BANGET, ADA CONTOH ESSAYSNYA JUGA!\n\nPenjelasannya enak, dan bisa banget buat naikin skor TOEFL kamu!'},
 {'id': 1,
  'text': 'Sekumpulan doktor pakar onkologi berjaya menyembuhkan kanser rektum menggunakan sejenis ubat yang masih di peringkat ujian klinikal, lapor akhbar Metro UK, semalam.'},
 {'id': 2,
  'text': 'Dua daripada empat suspek kes samun sebuah klinik pergigian di Taman Melati, Kuala Lumpur pada pukul 6.30 petang semalam berjaya diberkas pihak polis.'},
 {'id': 3,
  'text': 'Ngga siap liat coach naga api ketemu coach naga air di lapangan, makin seru. Selamaat coach'},
 {'id': 4,
  'text': 'Pengeluar dan peruncit barangan kemas terkemuka, Kumpulan HABIB mengorak langkah membangunkan sebuah masjid yang bakal menjadi pusat ibadah dan komuniti di Bandar Tanjong Pinang, Tanjong Tokong, di sini.'}]

You see that `test.jsonl` has a similar format, except it has no `class` key.

## Your task

Using the data in `train.jsonl`, use any classifier to generate class predictions of 0 and 1 for the text in `test.jsonl`. Write your output to this format below, and submit it to our competition server using the provided `post_answer` function.

```python
answer = [
  {'id': 0, 'class': 0},
  {'id': 1, 'class': 0},
  {'id': 2, 'class': 0},
  {'id': 3, 'class': 0},
  {'id': 4, 'class': 0},
  {'id': 5, 'class': 0},
  ...
]

post_answer({"solution": answer})
```

Make sure your answer has similar format to this sample answer: 

```python
# Note: this sample answer if submitted will return a score of 0 due to how F1 score works! Do you know why?
sample_answer = [{"id": i, "class": 0} for i in range(len(test))]
```

Remember to only produce predictions that are either 0 or 1!

This challenge will be graded primarily via API submission. Scoring as follows:

- Up to 10 pts for model performance, F1 score X 10. F1 score will be calculated via the autograder. 10 submissions allowed. Last submission will be used for scoring.
- Please submit your notebook as well! Random notebooks will be inspected to verify API submission results. In addition, partial credit may be granted for incomplete work at discretion



```
EXPLANATION:

Firstly, I have to convert classes 1, 2, 3, 4, 5 in the train dataset into '1' (a binary label for all bahasa pasar)

Then I used TF-IDF Vectoriser to evaluate the importance of a word in a sentence relative to a bunch of other sentences.

I used Logistics Regression because its output is only either 0 or 1 so it can perform very well in linear tasks.
```

In [101]:
# Your work below
import urllib.request
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
from sklearn.metrics import f1_score, confusion_matrix, classification_report

urllib.request.urlretrieve(
    "https://storage.googleapis.com/aiolympiadmy/baku-or-pasar/train.jsonl", 
    "train.jsonl"
)
urllib.request.urlretrieve(
    "https://storage.googleapis.com/aiolympiadmy/baku-or-pasar/test.jsonl", 
    "test.jsonl"
)

with open("train.jsonl", "r") as f:
    train = [json.loads(line) for line in f]
    X_train_full = [item["text"] for item in train]
    y_train_full = [0 if item["class"] == 0 else 1 for item in train]

with open("test.jsonl", "r") as f:
    test = [json.loads(line) for line in f]
    X_test = [item["text"] for item in test]
    test_ids = [item["id"] for item in test]

model = make_pipeline(
    TfidfVectorizer(),
    LogisticRegression(max_iter=1000, random_state=42)
)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, X_train_full, y_train_full, cv=cv, scoring='f1')

print(f"Cross-validated F1 scores: {[round(s, 4) for s in cv_scores]}")
print(f"Mean F1: {round(cv_scores.mean(), 4)} (±{round(cv_scores.std(), 4)})")

X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, 
    test_size=0.2, 
    stratify=y_train_full,
    random_state=42
)

model.fit(X_train, y_train)
val_preds = model.predict(X_val)

print("\nValidation set performance:")
print(f"F1 Score: {round(f1_score(y_val, val_preds), 4)}")
print("Confusion Matrix:")
print(confusion_matrix(y_val, val_preds))
print("\nClassification Report:")
print(classification_report(y_val, val_preds, digits=4))

# Retrain on full data
model.fit(X_train_full, y_train_full)

test_preds = model.predict(X_test)

answer = [{"id": tid, "class": int(pred)} for tid, pred in zip(test_ids, test_preds)]

print(json.dumps(answer))

Cross-validated F1 scores: [np.float64(0.975), np.float64(0.9811), np.float64(0.975), np.float64(0.9811), np.float64(0.9816)]
Mean F1: 0.9788 (±0.0031)

Validation set performance:
F1 Score: 0.9939
Confusion Matrix:
[[82  1]
 [ 0 81]]

Classification Report:
              precision    recall  f1-score   support

           0     1.0000    0.9880    0.9939        83
           1     0.9878    1.0000    0.9939        81

    accuracy                         0.9939       164
   macro avg     0.9939    0.9940    0.9939       164
weighted avg     0.9940    0.9939    0.9939       164

[{"id": 0, "class": 1}, {"id": 1, "class": 0}, {"id": 2, "class": 0}, {"id": 3, "class": 1}, {"id": 4, "class": 0}, {"id": 5, "class": 1}, {"id": 6, "class": 0}, {"id": 7, "class": 1}, {"id": 8, "class": 1}, {"id": 9, "class": 1}, {"id": 10, "class": 0}, {"id": 11, "class": 1}, {"id": 12, "class": 1}, {"id": 13, "class": 0}, {"id": 14, "class": 1}, {"id": 15, "class": 1}, {"id": 16, "class": 1}, {"id": 17, "cla

In [102]:
post_answer({"solution": answer})

{'status': 'SUCCESS',
 'message': 'Answer for challenge maio_2025_baku_or_pasar submitted successfully on 2025-03-16 11:59:52.153550+00:00. Total submissions is 9 / 10.'}